In [ ]:
elastic_rods_dir = '../elastic_rods/python/'
weaving_dir = './'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, optimization_visualization_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
importlib.reload(optimization_visualization_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint, write_distance_to_linkage_mesh, get_fixed_boundary_joint, set_design_parameters_from_topology, get_max_distance_to_target_surface)
import vis.fields
import matplotlib.cm as cm
import time
from optimization_visualization_helper import (compute_visualization_data_from_raw_data, get_objective_components_stage1, get_objective_components_stage2, get_objective_components_stage3, set_figure_label_and_limit, Visualization_Setting, plot_objective, plot_ribbon_component_analysis, render_video, combine_three_stage_video)


In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
rod_length = 0.21162395234664752
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
# width *= 0.5
# thickness *= 0.5

io = InputOrganizer('simple_hemoglobin_5_1_z4.6', thickness, width, weaving_dir)
#  [ ear 214, ear 255, noseZ 39]

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 2000
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
import pickle 
import gzip

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type = InterleavingType.weaving)
input_joint_normals = np.reshape(curved_linkage.get_closest_point_normal(curved_linkage.jointPositions()), (curved_linkage.numJoints(), 3))
#         current_joint_normals = [curved_linkage.joint(i).normal for i in range(curved_linkage.numJoints())]
#         current_joint_normals[35] = input_joint_normals[35]
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, use_constant_width=True, interleaving_type=InterleavingType.triaxialWeave, input_joint_normals = input_joint_normals)

In [ ]:
set_design_parameters_from_topology(curved_linkage, io)
curved_linkage.attraction_weight = 1e-5
with so(): elastic_rods.compute_equilibrium(curved_linkage, callback = None, options = OPTS, fixedVars = [])
distance_to_target_initialization = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)

In [ ]:
distance_to_target_initialization

In [ ]:
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
curved_linkage_view.show()

In [ ]:
# with so(): curved_linkage = initialize_linkage(surface_path = None, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION)
# curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
# curved_save_tgt_joint_pos = curved_linkage.jointPositions()
curved_linkage = pickle.load(gzip.open('mega_monster_optimization_diagram_results/bunny_head_small_triaxial_1_stage_3.pkl.gz', 'r'))
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view, linkage_color='lightgreen')
curved_linkage_view.show()

In [ ]:
curved_linkage.totalRestLength()

In [ ]:
curved_linkage.totalRestLength()

In [ ]:
distance_to_target_initialization = np.round(get_max_distance_to_target_surface(curved_linkage) / curved_linkage.get_l0(), 4)

In [ ]:
distance_to_target_initialization

In [ ]:
fix_boundary = False

In [ ]:
curved_linkage.attraction_weight = 1e-5
fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
curved_linkage_view.update()

In [ ]:
renderCam = curved_linkage_view_2.getCameraParams()

In [ ]:
np.save('kleinbottle_renderCam.npy', renderCam)

In [ ]:
renderCam = np.load('kleinbottle_renderCam.npy')

In [ ]:
def renderToFile(view, renderCam, path):
    orender = view.offscreenRenderer(width=2048, height=2048)
    orender.setCameraParams(renderCam)
    orender.render()
    orender.save(path)

In [ ]:
distance_color = write_distance_to_linkage_mesh(curved_linkage_2, max(io.RIBBON_CS), None, return_distance_field = True)
curved_linkage_view_2.update(scalarField = distance_color[:, :3])
renderToFile(curved_linkage_view_2, renderCam, 'manual_klein.png')

In [ ]:
scale = 1
rod_length = 0.07244622391567243 * scale
width = rod_length / 20 * 10 
thickness = width / 10 * 0.35 
print(width, thickness)
io = InputOrganizer('kleinbottle_projected_{}'.format(scale), thickness, width, weaving_dir)
with so(): curved_linkage_2 = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving, use_constant_width = False, width_scale = [0.5, 1.5])
curved_linkage_2.set_design_parameter_config(use_restLen = True, use_restKappa = True)
# curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view_2 = linkage_vis.LinkageViewerWithSurface(curved_linkage_2, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view_2)
curved_linkage_view_2.show()

In [ ]:
curved_linkage_2.attraction_weight = 1e-5
set_design_parameters_from_topology(curved_linkage_2, io)
elastic_rods.compute_equilibrium(curved_linkage_2, callback = None, options = OPTS, fixedVars = [])

In [ ]:
curved_linkage_view_2.update()

In [ ]:
distance_to_target_initialization = np.round(get_max_distance_to_target_surface(curved_linkage_2) / curved_linkage_2.get_l0(), 4)

In [ ]:
distance_to_target_initialization

In [ ]:
structure_view = get_structure_analysis_view(curved_linkage)
structure_view.show()

In [ ]:
from structural_analysis import weavingCrossingForceMagnitudes
cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
np.percentile(cfm[:, 0], 75), -np.percentile(np.abs(cfm[:, 0]), 10)

In [ ]:
structure_view_2 = get_structure_analysis_view(curved_linkage)
structure_view_2.show()

In [ ]:
def get_opposite_joint_type(curr_type):
    if curr_type == curved_linkage.joint(0).Type.A_OVER_B:
        return curved_linkage.joint(0).Type.B_OVER_A
    return curved_linkage.joint(0).Type.A_OVER_B
import structural_analysis

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
second_structure_view = get_structure_analysis_view(curved_linkage)
second_structure_view.show()

In [ ]:
# import pickle
# import gzip
# pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210105_after_flipping_1.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
third_structure_view = get_structure_analysis_view(curved_linkage)
third_structure_view.show()

In [ ]:
# import pickle
# import gzip
# pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210105_after_flipping_2.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
fourth_structure_view = get_structure_analysis_view(curved_linkage)
fourth_structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
# import pickle
# import gzip
# pickle.dump(curved_linkage, gzip.open('{}_stage_2_20210105_after_flipping_3.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
fifth_structure_view = get_structure_analysis_view(curved_linkage)
fifth_structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
sixth_structure_view = get_structure_analysis_view(curved_linkage)
sixth_structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
seventh_structure_view = get_structure_analysis_view(curved_linkage)
seventh_structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
eighth_structure_view = get_structure_analysis_view(curved_linkage)
eighth_structure_view.show()

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
ninth_structure_view = get_structure_analysis_view(curved_linkage)
ninth_structure_view.show()

In [ ]:
import pickle
import gzip
pickle.dump(curved_linkage, gzip.open('bunny_stage_3_20210112_after_flipping_8.pkl.gz', 'w'))

In [ ]:
cfm = structural_analysis.weavingCrossingForceMagnitudes(curved_linkage, True)
if np.sum(cfm[:, 0]>0) > 0:
    print("Flipping crossing: ", np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):])
    for i in np.argsort(cfm[:, 0] > 0)[-1 * np.sum(cfm[:, 0] > 0):]:
        curved_linkage.joint(i).type = get_opposite_joint_type(curved_linkage.joint(i).type)
    fixed_boundary_joints = get_fixed_boundary_joint(curved_linkage) if fix_boundary else []
    def eqm_callback(prob, i):
        pass
    elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = fixed_boundary_joints)
    curved_linkage_view.update()

In [ ]:
tenth_structure_view = get_structure_analysis_view(curved_linkage)
tenth_structure_view.show()

In [ ]:
mega_monster_optimization_diagram_results/fifth_round/bunny_head_small_triaxial_1/bunny_head_small_triaxial_1_stage_3.pkl.gz

### Visualize Force Distribution

In [ ]:
data = np.load('mega_monster_optimization_diagram_results/kleinbottle_projected_1_stage_2.npy', allow_pickle = True)

In [ ]:
sum([pt['iteration_time'] for pt in data])

In [ ]:
from structural_analysis import weavingCrossingForceMagnitudes
cfm = weavingCrossingForceMagnitudes(curved_linkage, True)
print("Normal Activation Threshold: ", np.percentile(cfm[:, 0], 75))

In [ ]:
import seaborn as sns

In [ ]:
separationForce = cfm[:, 0]

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 4, 1)
plt.xlim((separationForce.min(), max([0, separationForce.max()])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Separation Forces')
plt.xlabel('Separation Force Mag.')
plt.ylabel('Number of Crossings')
plt.hist(separationForce, 200);

In [ ]:
curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/2_28.pkl.gz', 'r'))
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
from structural_analysis import weavingCrossingForceMagnitudes
stage_2_cfm = weavingCrossingForceMagnitudes(curved_linkage, True)


curved_linkage = pickle.load(gzip.open('optimization_diagram_results/simple_hemoglobin_5_1/pickle/3_34.pkl.gz', 'r'))
curved_linkage.attraction_weight = 1e-5
def eqm_callback(prob, i):
    pass
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)
from structural_analysis import weavingCrossingForceMagnitudes
stage_3_cfm = weavingCrossingForceMagnitudes(curved_linkage, True)

In [ ]:
stage_2_separation = stage_2_cfm[:, 0]
stage_3_separation = stage_3_cfm[:, 0]

In [ ]:
min_separation = min(min(stage_2_separation), min(stage_3_separation))
max_separation = max(max(stage_2_separation), max(stage_3_separation))

In [ ]:
import pandas as pd

In [ ]:
fig, host = plt.subplots()
data = np.transpose([stage_2_separation, stage_3_separation])
data = pd.DataFrame(data, columns=['Stage 2', 'Stage 3'])
sns.kdeplot(data = data, shade = True, cut = True)
plt.xlim((min_separation, max([0, max_separation])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Density of Separation Force Mag. at Crossings', fontsize = 14)
plt.xlabel('Separation Force Mag.', fontsize=14); 
plt.ylabel('Density', fontsize=14);
fig.set_size_inches((10, 4))
plt.tight_layout()
host.get_yaxis().set_ticks([])
plt.savefig('./separation_force_mag.png', format='png', dpi = 200)

In [ ]:
stage_2_tangential = stage_2_cfm[:, 1]
stage_3_tangential = stage_3_cfm[:, 1]

In [ ]:
min_tangential = min(min(stage_2_tangential), min(stage_3_tangential))
max_tangential = max(max(stage_2_tangential), max(stage_3_tangential))

In [ ]:
import pandas as pd

In [ ]:
fig, host = plt.subplots()
data = np.transpose([stage_2_tangential, stage_3_tangential])
data = pd.DataFrame(data, columns=['Stage 2', 'Stage 3'])
sns.kdeplot(data = data, shade = True, cut = True)
plt.xlim((min_tangential, max([0, max_tangential])))
plt.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
plt.title('Density of Tangential Force Mag. at Crossings', fontsize = 14)
plt.xlabel('Tangential Force Mag.', fontsize=14); 
plt.ylabel('Density', fontsize=14);
fig.set_size_inches((10, 4))
plt.tight_layout()
host.get_yaxis().set_ticks([])
plt.savefig('./tangential_force_mag.png', format='png', dpi = 200)

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, 'sphere' + '_stage_2', 'sphere' + '_stage_2', False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
os.chdir(save_path)